# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from semeval_reader import SemevalReader

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from torch import cuda

from sa_models.SA_BERT_Dropout_CNN_BiLSTM_Linear import SA_BERT_Dropout_CNN_BiLSTM_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
def get_target_list_for_polarity(polarity):
    if polarity == 'positive':
        return [0, 0, 1]
    if polarity == 'negative':
        return [1, 0, 0]
    return [0, 1, 0]

In [7]:
semeval_reader = SemevalReader('../../../data/semeval16_restaurants_train.xml')

reviews = semeval_reader.read_reviews()
absolute_polarity_sentences = semeval_reader.get_absolute_polarity_sentences()

df = pd.DataFrame(map(lambda x: (x.text, x.opinions[0].polarity), absolute_polarity_sentences))
df.rename(columns={0: 'text'}, inplace=True)
df['target_list'] = df.apply(lambda row: get_target_list_for_polarity(row[1]), axis=1)

absolute_polarity_df = df.drop(columns=[1])

# Train & Validate

In [8]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [9]:
BERT_FINE_TUNED_PATH = '../../../results/SA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned.pth'

In [10]:
MODEL_OUTPUT = '../../../results/SA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned_dropout_cnn_bilstm_linear.pth'
STATS_OUTPUT = '../../../results/SA/SemEval16 - Task 5 - Restaurants/stats/bert_fine_tuned_dropout_cnn_bilstm_linear.csv'

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [12]:
best_accuracy = 0.0

In [13]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [14]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)
            
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    return fin_outputs, fin_targets

In [15]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [16]:
for i in range(10):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/10")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True
    )

    model = SA_BERT_Dropout_CNN_BiLSTM_Linear(bert=torch.load(BERT_FINE_TUNED_PATH), bert_seq_len=512, dropout=0.3, bilstm_in_features=256, no_out_labels=3, conv_out_channels=54, conv_kernel_size=3, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.BCEWithLogitsLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    outputs = np.argmax(outputs, axis=1)
    targets = np.argmax(targets, axis=1)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6911834478378296
Epoch: 0/2, Batch: 31/313, Loss: 0.4908279776573181
Epoch: 0/2, Batch: 62/313, Loss: 0.24622946977615356
Epoch: 0/2, Batch: 93/313, Loss: 0.09542150795459747
Epoch: 0/2, Batch: 124/313, Loss: 0.1049174889922142
Epoch: 0/2, Batch: 155/313, Loss: 0.2995476722717285
Epoch: 0/2, Batch: 186/313, Loss: 0.05574151501059532
Epoch: 0/2, Batch: 217/313, Loss: 0.022928528487682343
Epoch: 0/2, Batch: 248/313, Loss: 0.058456242084503174
Epoch: 0/2, Batch: 279/313, Loss: 0.72101229429245
Epoch: 0/2, Batch: 310/313, Loss: 0.037716519087553024
Epoch: 1/2, Batch: 0/313, Loss: 0.019887493923306465
Epoch: 1/2, Batch: 31/313, Loss: 0.04054143279790878
Epoch: 1/2, Batch: 62/313, Loss: 0.2504020631313324
Epoch: 1/2, Batch: 93/313, Loss: 0.027925487607717514
Epoch: 1/2, Batch: 124/313, Loss: 0.03899519890546799
Epoch: 1/2, Batch: 155/313, Loss: 0.04766596853733063
Epoch: 1/2, Batch: 186/313, Loss: 0.022837353870272636
Epoch: 1/2, Batch: 217/313, Los

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 2/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6981625556945801
Epoch: 0/2, Batch: 31/313, Loss: 0.6450072526931763
Epoch: 0/2, Batch: 62/313, Loss: 0.3166102170944214
Epoch: 0/2, Batch: 93/313, Loss: 0.14344081282615662
Epoch: 0/2, Batch: 124/313, Loss: 0.08628766238689423
Epoch: 0/2, Batch: 155/313, Loss: 0.059973813593387604
Epoch: 0/2, Batch: 186/313, Loss: 0.08780910819768906
Epoch: 0/2, Batch: 217/313, Loss: 0.04029981419444084
Epoch: 0/2, Batch: 248/313, Loss: 0.035028621554374695
Epoch: 0/2, Batch: 279/313, Loss: 0.01427082996815443
Epoch: 0/2, Batch: 310/313, Loss: 0.015102590434253216
Epoch: 1/2, Batch: 0/313, Loss: 0.39837950468063354
Epoch: 1/2, Batch: 31/313, Loss: 0.05869857594370842
Epoch: 1/2, Batch: 62/313, Loss: 0.03177323192358017
Epoch: 1/2, Batch: 93/313, Loss: 0.01741618849337101
Epoch: 1/2, Batch: 124/313, Loss: 0.01989118568599224
Epoch: 1/2, Batch: 155/313, Loss: 0.010914698243141174
Epoch: 1/2, Batch: 186/313, Loss: 0.007516535930335522
Epoch: 1/2, Batch: 217/313,

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 3/10
Epoch: 0/2, Batch: 0/313, Loss: 0.695098340511322
Epoch: 0/2, Batch: 31/313, Loss: 0.5463918447494507
Epoch: 0/2, Batch: 62/313, Loss: 0.24131831526756287
Epoch: 0/2, Batch: 93/313, Loss: 0.1241360455751419
Epoch: 0/2, Batch: 124/313, Loss: 0.07287150621414185
Epoch: 0/2, Batch: 155/313, Loss: 0.167068213224411
Epoch: 0/2, Batch: 186/313, Loss: 0.056662943214178085
Epoch: 0/2, Batch: 217/313, Loss: 0.0479985736310482
Epoch: 0/2, Batch: 248/313, Loss: 0.46262091398239136
Epoch: 0/2, Batch: 279/313, Loss: 0.40072426199913025
Epoch: 0/2, Batch: 310/313, Loss: 0.015038315206766129
Epoch: 1/2, Batch: 0/313, Loss: 0.035558752715587616
Epoch: 1/2, Batch: 31/313, Loss: 0.03163919597864151
Epoch: 1/2, Batch: 62/313, Loss: 0.025168027728796005
Epoch: 1/2, Batch: 93/313, Loss: 0.03797636926174164
Epoch: 1/2, Batch: 124/313, Loss: 0.026507645845413208
Epoch: 1/2, Batch: 155/313, Loss: 0.030826382339000702
Epoch: 1/2, Batch: 186/313, Loss: 0.008252881467342377
Epoch: 1/2, Batch: 217/313, L

...

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 8/10
Epoch: 0/2, Batch: 0/313, Loss: 0.7027940154075623
Epoch: 0/2, Batch: 31/313, Loss: 0.4189358651638031
Epoch: 0/2, Batch: 62/313, Loss: 0.13307060301303864
Epoch: 0/2, Batch: 93/313, Loss: 0.3253505825996399
Epoch: 0/2, Batch: 124/313, Loss: 0.2479211986064911
Epoch: 0/2, Batch: 155/313, Loss: 0.09518089145421982
Epoch: 0/2, Batch: 186/313, Loss: 0.05477684363722801
Epoch: 0/2, Batch: 217/313, Loss: 0.0805746540427208
Epoch: 0/2, Batch: 248/313, Loss: 0.06415490806102753
Epoch: 0/2, Batch: 279/313, Loss: 0.03441053628921509
Epoch: 0/2, Batch: 310/313, Loss: 0.043582484126091
Epoch: 1/2, Batch: 0/313, Loss: 0.05861007422208786
Epoch: 1/2, Batch: 31/313, Loss: 0.03248023986816406
Epoch: 1/2, Batch: 62/313, Loss: 0.019132563844323158
Epoch: 1/2, Batch: 93/313, Loss: 0.015969878062605858
Epoch: 1/2, Batch: 124/313, Loss: 0.035761937499046326
Epoch: 1/2, Batch: 155/313, Loss: 0.01431641262024641
Epoch: 1/2, Batch: 186/313, Loss: 0.012476949952542782
Epoch: 1/2, Batch: 217/313, Loss

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 9/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6883329153060913
Epoch: 0/2, Batch: 31/313, Loss: 0.46554893255233765
Epoch: 0/2, Batch: 62/313, Loss: 0.34840965270996094
Epoch: 0/2, Batch: 93/313, Loss: 0.6002908945083618
Epoch: 0/2, Batch: 124/313, Loss: 0.3717585504055023
Epoch: 0/2, Batch: 155/313, Loss: 0.12101833522319794
Epoch: 0/2, Batch: 186/313, Loss: 0.031506169587373734
Epoch: 0/2, Batch: 217/313, Loss: 0.04203178733587265
Epoch: 0/2, Batch: 248/313, Loss: 0.035717424005270004
Epoch: 0/2, Batch: 279/313, Loss: 0.015859972685575485
Epoch: 0/2, Batch: 310/313, Loss: 0.0271746963262558
Epoch: 1/2, Batch: 0/313, Loss: 0.2756875157356262
Epoch: 1/2, Batch: 31/313, Loss: 0.020368728786706924
Epoch: 1/2, Batch: 62/313, Loss: 0.013878196477890015
Epoch: 1/2, Batch: 93/313, Loss: 0.00835687667131424
Epoch: 1/2, Batch: 124/313, Loss: 0.01379365473985672
Epoch: 1/2, Batch: 155/313, Loss: 0.9638628959655762
Epoch: 1/2, Batch: 186/313, Loss: 0.01711626723408699
Epoch: 1/2, Batch: 217/313, Lo

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.913462,0.913462,0.614753,0.913462,0.620072,0.913462,0.615983,200.170856
1,0.939103,0.939103,0.620478,0.939103,0.645700,0.939103,0.632837,196.963544
2,0.942308,0.942308,0.611399,0.942308,0.650989,0.942308,0.629808,201.066161
3,0.939103,0.939103,0.620880,0.939103,0.641971,0.939103,0.631159,199.399690
4,0.935897,0.935897,0.947168,0.935897,0.670693,0.935897,0.676560,197.278369
5,0.942308,0.942308,0.960061,0.942308,0.702043,0.942308,0.741949,196.999322
6,0.951923,0.951923,0.617509,0.951923,0.659125,0.951923,0.636590,197.067465
7,0.935897,0.935897,0.621418,0.935897,0.631392,0.935897,0.626257,197.309085
8,0.945513,0.945513,0.959417,0.945513,0.676304,0.945513,0.688389,197.303443
9,0.961538,0.961538,0.633063,0.961538,0.656958,0.961538,0.644258,197.120684


In [18]:
results.to_csv(STATS_OUTPUT)